In [ ]:
This jupyter notebook consists of all the codes and the purpose of each code.
Please dont run the codes as it will give you error messages.
To run the codes you can refer the "JUPYTER ANALYSIS NOTEBOOK.ipynb" file.

In [ ]:
### **1. Libraries and Data Loading**

import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

- **Purpose**: Imports libraries for:
  - `pandas`: Data manipulation.
  - `numpy`: Numerical operations.
  - `re`: Regular expressions for string processing.
  - `datetime`: Date handling.
  - `matplotlib/seaborn`: Plotting.
  - `scipy.stats`: Statistical tests (e.g., t-tests).

In [ ]:
### **2. Load Datasets**
transactions = pd.read_csv(file_path + "QVI_transaction_data.csv")
customers = pd.read_csv(file_path + "QVI_purchase_behaviour.csv")

- **Purpose**: Load transaction and customer data from CSV files.
- **Note**: Replace `file_path` with your actual directory path.

In [ ]:
### **3. Convert DATE Column**
transactions['DATE'] = pd.to_datetime(transactions['DATE'], origin='1899-12-30', unit='D')

- **What it does**:
  - Converts integer dates (e.g., `43390`) to actual dates (e.g., `2018-10-17`).
  - origin='1899-12-30': Excel’s date system starts from this date.
  - unit='D': Treats the integer as days since the origin.

In [ ]:
### **4. Product Analysis**
#### **Extract Words from Product Names**
product_words = transactions['PROD_NAME'].str.split().explode().reset_index()

- **str.split()**: Splits product names into lists of words.  
  Example: "Natural Chip SeaSalt" → ["Natural", "Chip", "SeaSalt"].
- **explode()**: Converts lists into individual rows.  
  Example: A row with ["Natural", "Chip"] becomes two rows: "Natural" and "Chip".
- **reset_index()**: Resets the index to retain the original row numbers.

In [ ]:
#### **Clean Words**
product_words = product_words[~product_words['words'].str.contains(r'\d', na=False)]
product_words = product_words[product_words['words'].str.isalpha()]

- **~product_words['words'].str.contains(r'\d')**: Drops words containing digits (e.g., `175g`).
- **str.isalpha()**: Keeps only alphabetic words (removes special characters).

In [ ]:
### **5. Remove Salsa Products**
transactions = transactions[~transactions['PROD_NAME'].str.lower().str.contains('salsa')]

- **Purpose**: Filters out rows where `PROD_NAME` contains "salsa".
- **str.lower()**: Converts names to lowercase for case-insensitive matching.

In [ ]:
### **6. Handle Outliers**
outlier_customer = transactions[transactions['PROD_QTY'] == 200]['LYLTY_CARD_NBR'].values[0]
transactions = transactions[transactions['LYLTY_CARD_NBR'] != outlier_customer]

- **Purpose**: Removes a customer who bought 200 packets (likely a commercial buyer).
- **values[0]**: Extracts the loyalty card number of the outlier.
- **transactions[... != ...]**: Filters out all transactions from this customer.

In [ ]:
### **7. Create Pack Size Feature**
transactions['PACK_SIZE'] = transactions['PROD_NAME'].str.extract(r'(\d+)').astype(float)

- **str.extract(r'(\d+)')**: Extracts the first sequence of digits from `PROD_NAME` (e.g., `175g` → `175`).
- **astype(float)**: Converts the extracted string to a numeric value.

In [ ]:
### **8. Create Brand Feature**
transactions['BRAND'] = transactions['PROD_NAME'].str.split().str[0].str.upper()

- **str.split().str[0]**: Takes the first word of the product name (e.g., "Kettle SeaSalt" → "Kettle").
- **str.upper()**: Converts the brand name to uppercase for consistency.

In [ ]:
### **9. Clean Brand Names**
brand_replacements = {'RED': 'RRD', 'SNBTS': 'SUNBITES', ...}
transactions['BRAND'] = transactions['BRAND'].replace(brand_replacements)

- **Purpose**: Consolidates variations of brand names (e.g., `"RED"` → `"RRD"`).

In [ ]:
### **10. Merge Datasets**
data = pd.merge(transactions, customers, on='LYLTY_CARD_NBR', how='left')

- **on='LYLTY_CARD_NBR'**: Joins on the loyalty card number.
- **how='left'**: Keeps all rows from `transactions` (equivalent to R’s `all.x = TRUE`).

In [ ]:
### **11. Analysis Functions**
def calculate_metrics(df):
    return pd.DataFrame({
        'SALES': df['TOT_SALES'].sum(),
        'CUSTOMERS': df['LYLTY_CARD_NBR'].nunique(),
        'AVG_UNITS': df['PROD_QTY'].sum() / df['LYLTY_CARD_NBR'].nunique(),
        'AVG_PRICE': df['TOT_SALES'].sum() / df['PROD_QTY'].sum()
    })

- **Purpose**: Calculates key metrics for each customer segment:
  - Total sales.
  - Number of unique customers.
  - Average units per customer.
  - Average price per unit.

In [ ]:
### **12. Group by Segments**
segment_analysis = data.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER']).apply(calculate_metrics).reset_index()

- **groupby()**: Groups data by `LIFESTAGE` and `PREMIUM_CUSTOMER`.
- **apply(calculate_metrics)**: Applies the metric calculation to each group.

In [ ]:
### **13. Plot Total Sales**
sns.barplot(x='LIFESTAGE', y='SALES', hue='PREMIUM_CUSTOMER', data=segment_analysis)

- **Purpose**: Visualizes total sales by customer segments.
- **hue='PREMIUM_CUSTOMER'**: Splits bars by premium status.

In [ ]:
### **14. T-Test Implementation**
t_stat, p_value = stats.ttest_ind(mainstream.dropna(), others.dropna())

- **Purpose**: Tests if the average price paid by mainstream customers is significantly higher than others.
- **ttest_ind()**: Independent t-test (compares two groups).

In [ ]:
### **15. Brand Affinity Analysis**

brand_affinity = (segment.groupby('BRAND')['PROD_QTY'].sum() / segment_total) / (others.groupby('BRAND')['PROD_QTY'].sum() / others_total)
```
- **Purpose**: Compares the proportion of brand purchases for a target segment vs. others.
- **Example**: A value of `1.23` means the segment is 23% more likely to buy that brand.